In [1]:
%%capture
!pip install transformers

In [2]:
import torch
from transformers import AutoTokenizer,BertTokenizerFast

#載入fine tuned model

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Define the bert tokenizer
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Load the fine-tuned model
model = torch.load("/content/drive/MyDrive/讀書會/bert/SQuAD20/test_model",map_location=torch.device('cpu'))
model.eval()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

#測試model
https://developer.apple.com/documentation/coreml/model_integration_samples/finding_answers_to_questions_in_a_text_document

In [10]:
def predict(context,query):

  inputs = tokenizer.encode_plus(query, context, return_tensors='pt')
  
  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0]) 
  answer_end = torch.argmax(outputs[1]) + 1 
  # print(inputs)
  # print(outputs)
  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  if answer[-1] == '.':
    return answer[:-1]

  return answer

In [11]:
def give_an_answer(context,query,answer):

  prediction = predict(context,query)

  print(f"Question: {query}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print("\n")

In [12]:
context = """Gigi is a student in NLP lab. She lives in Taichung."""

queries = ["Where does Gigi live?"]
answers = ["Taichung"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: Where does Gigi live?
Prediction: taichung
True Answer: Taichung




In [13]:
context = """ Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. 
          The novels chronicle the lives of a young wizard, Harry Potter, and his friends Hermione 
          Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. 
          The main story arc concerns Harry's struggle against Lord Voldemort, a dark wizard who intends to 
          become immortal, overthrow the wizard governing body known as the Ministry of Magic and subjugate 
          all wizards and Muggles (non-magical people). """

queries = ["Who is the author of harry potter?"]
answers = ["J. K. Rowling"]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: Who is the author of harry potter?
Prediction: j. k. rowling
True Answer: J. K. Rowling




In [14]:
context = """ Queen are a British rock band formed in London in 1970. Their classic line-up was Freddie Mercury (lead vocals, piano), 
            Brian May (guitar, vocals), Roger Taylor (drums, vocals) and John Deacon (bass). Their earliest works were influenced 
            by progressive rock, hard rock and heavy metal, but the band gradually ventured into more conventional and radio-friendly 
            works by incorporating further styles, such as arena rock and pop rock. """

queries = ["When did Queen found?",
           "Who were the basic members of Queen band?",
           "What kind of band they are?"
          ]
answers = ["1970",
           "Freddie Mercury, Brian May, Roger Taylor and John Deacon",
           "rock"
          ]

for q,a in zip(queries,answers):
  give_an_answer(context,q,a)

Question: When did Queen found?
Prediction: 1970
True Answer: 1970


Question: Who were the basic members of Queen band?
Prediction: freddie mercury ( lead vocals, piano ), brian may
True Answer: Freddie Mercury, Brian May, Roger Taylor and John Deacon


Question: What kind of band they are?
Prediction: rock
True Answer: rock


